In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import random
import pdb

### Make a list of the urls of the categories in the forum

In [2]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://curltalk.naturallycurly.com/', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
link_list_forum_top = []
for link in soup.find_all('a'):
    link_list_forum_top.append(link.get('href'))
categories = [s for s in link_list_forum_top if (("categories" in s) and ('https' in s))]
categories[0:4]

['https://curltalk.naturallycurly.com/categories/announcements',
 'https://curltalk.naturallycurly.com/categories/newbies-info',
 'https://curltalk.naturallycurly.com/categories/introductions',
 'https://curltalk.naturallycurly.com/categories/curlmart-news']

#### Get the discussion topic urls for the first x pages of the category

In [3]:
# %%time

link_listdiscussion = []
for i in range(1, 11):
    url = f'{categories[5]}/p{i}'
    print(url)
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
     
    # Check status code for an appropriate response fromt the API
    status = html_page.status_code
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    page1_of_discussion = []
    time.sleep(random.randint(15,30))
    for link in soup.find_all('a'):
        link_listdiscussion.append(link.get('href'))
        topics = [s for s in link_listdiscussion if (("/discussion" in s) and ('https' in s))]

print(len(topics))     
# Loop through all the topics found for each catagory        
list_for_mongo = []
count = 1
for topic in topics:
    url2 = topic
    user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    html_page2 = requests.get(url2, headers = {'User-Agent': user_agent} )
    
    if status != 200:
        print(f'Error improper response code. Code is {status}')
        break
        
    soup2 = BeautifulSoup(html_page2.content, 'html.parser')
    signatures = soup2.find_all('div', class_="Signature UserSignature userContent")
    
    for i in range(0,len(signatures)):
        sig = {}
        sig['signature'] = soup2.find_all('div', class_="Signature UserSignature userContent")[i].get_text()
        list_for_mongo.append(sig)
    time.sleep(random.randint(20,60))
    print(count)
    count += 1
    

https://curltalk.naturallycurly.com/categories/c2/p1
https://curltalk.naturallycurly.com/categories/c2/p2
https://curltalk.naturallycurly.com/categories/c2/p3
https://curltalk.naturallycurly.com/categories/c2/p4
https://curltalk.naturallycurly.com/categories/c2/p5
https://curltalk.naturallycurly.com/categories/c2/p6
https://curltalk.naturallycurly.com/categories/c2/p7
https://curltalk.naturallycurly.com/categories/c2/p8
https://curltalk.naturallycurly.com/categories/c2/p9
https://curltalk.naturallycurly.com/categories/c2/p10
300
1
2
3
4
5
6
7
8
9
10
11
12
13


ConnectionError: HTTPSConnectionPool(host='curltalk.naturallycurly.com', port=443): Max retries exceeded with url: /discussion/188004/hair-looks-greasy-how-do-i-fix-this (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1105736a0>: Failed to establish a new connection: [Errno 50] Network is down'))

In [4]:
list_for_mongo

[{'signature': "Animal obsessed wavy with a love for writing\xa02B/2C Indian wavy, low-medium porosity, medium-high density, coarse hairShampoo: WOW Miracle 10-in-1 ShampooConditioner: WOW Argan Oil ConditionerGel: Arata Hair GelDC/ Treatment: Shea Moisture Manuka Honey and Mafura Oil Intensive Hydration MasqueClarifying Shampoo: Pantene Lively CleanCG since May '18After nearly two decades of neglect, CG finally helped me embrace my hair as a part and parcel of my identity"},
 {'signature': "Animal obsessed wavy with a love for writing\xa02B/2C Indian wavy, low-medium porosity, medium-high density, coarse hairShampoo: WOW Miracle 10-in-1 ShampooConditioner: WOW Argan Oil ConditionerGel: Arata Hair GelDC/ Treatment: Shea Moisture Manuka Honey and Mafura Oil Intensive Hydration MasqueClarifying Shampoo: Pantene Lively CleanCG since May '18After nearly two decades of neglect, CG finally helped me embrace my hair as a part and parcel of my identity"},
 {'signature': "Formerly known as Jani

### Create a mongoDB and have data stored from scrape 

In [5]:
# DB creation 
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [6]:
entry2 = curly_collection.insert_many(list_for_mongo)

In [16]:
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])
    #print(x)
print(len(test))    

13440


In [18]:
test_set = set(test)

In [19]:
# Number of unique entries so far
len(test_set)

1377